In [10]:
from pvlib import location, irradiance
import requests
import pandas as pd
import os

In [18]:
# setup a solar plant in Berlin
berlin = location.Location(52.52, 13.405)
times = pd.date_range('2023-07-01', '2023-07-07', freq='H', tz='Europe/Berlin')
solpos = berlin.get_solarposition(times)

In [19]:
# assume fixed system
ghi = irradiance.get_total_irradiance(surface_tilt=30, surface_azimuth=180,
                                      solar_zenith=solpos['apparent_zenith'],
                                      solar_azimuth=solpos['azimuth'],
                                      dni=300, ghi=250, dhi=100)
ghi

,poa_global,poa_direct,poa_diffuse,poa_sky_diffuse,poa_ground_diffuse
2023-07-01 00:00:00+02:00,97.487976,0.0,97.487976,93.30127,4.186706
2023-07-01 01:00:00+02:00,97.487976,0.0,97.487976,93.30127,4.186706
2023-07-01 02:00:00+02:00,97.487976,0.0,97.487976,93.30127,4.186706
2023-07-01 03:00:00+02:00,97.487976,0.0,97.487976,93.30127,4.186706
2023-07-01 04:00:00+02:00,97.487976,0.0,97.487976,93.30127,4.186706
...,...,...,...,...,...
2023-07-06 20:00:00+02:00,97.487976,0.0,97.487976,93.30127,4.186706
2023-07-06 21:00:00+02:00,97.487976,0.0,97.487976,93.30127,4.186706
2023-07-06 22:00:00+02:00,97.487976,0.0,97.487976,93.30127,4.186706
2023-07-06 23:00:00+02:00,97.487976,0.0,97.487976,93.30127,4.186706


Saved German day-ahead prices to ../data/raw/prices.csv


In [23]:
# fetch hourly solar irradiance and wind speed using Open-Meteo API

def download_weather_data(lat, lon, start_date, end_date, save_path):
    lat = float(lat)
    lon = float(lon)
    
    url = (
        f"https://archive-api.open-meteo.com/v1/archive"
        f"?latitude={lat}&longitude={lon}"
        f"&start_date={start_date}&end_date={end_date}"
        f"&hourly=temperature_2m,windspeed_10m,shortwave_radiation"
        f"&timezone=Europe%2FBerlin"
    
    )

    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error URL: {url}")
        raise Exception(f"API request failed with status {response.status_code}")
    
    data = response.json()['hourly']
    df = pd.DataFrame(data)
    df['time'] = pd.to_datetime(df['time'])
    df = df.set_index('time')

    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    df.to_csv(save_path)
    print(f"Saved weather data to {save_path}")

if __name__ == "__main__":
    download_weather_data(
        lat=52.52, lon=13.405, #berlin
        start_date="2023-07-01", end_date="2023-07-07",
        save_path="../data/raw/weather_berlin.csv"
    )

Saved weather data to ../data/raw/weather_berlin.csv
